### Yelp web scraping

In [19]:
from bs4 import BeautifulSoup
import requests
import time
from urllib.request import Request, urlopen  
import re
import random
from pymongo import MongoClient
import pandas as pd

In [5]:
header1 = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
"authority": "www.tagesschau.de",
"method": "GET",
"path":"/",
"scheme":"https",
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9,de;q=0.8",
"cache-control": "max-age=0",
"cookie": "atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2257ea5dd6-4c35-4982-942f-8a7f8b8c3a4b%22%2C%22options%22%3A%7B%22end%22%3A%222023-02-17T05%3A02%3A59.936Z%22%2C%22path%22%3A%22%2F%22%7D%7D; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-595936-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D",
"sec-ch-ua-mobile": "?0",
"sec-ch-ua-platform": "Windows",
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "none",
"sec-fetch-user": "?1",
"upgrade-insecure-requests": "1"}

### Q1 Saving top 40 results - 4 pages

In [6]:
k = 1
for i in range(0,40,10):
            url = "https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA&ns=1&start="+str(i)
            r = requests.get(url,headers=header1)
            with open('sf_donut_shop_search_page_'+str(k)+'.htm', 'w') as file:
                file.write(r.text)
            time.sleep(2)
            k+=1

### Q2 Using Python or Java, write new code that opens the search result pages saved in (1) and parses out all shop information (search rank, name, linked URL [this store’s Yelp URL], star rating, number of reviews, store tags, $” signs, delivery / dine-in tags, and whether you can order through Yelp). Please be sure to skip all “Sponsored Results”.

In [7]:
ranking=[]
store_name=[]
url=[]
star=[]
review=[]
tag=[]
dollar=[]
deliver=[]
order=[]
for k in range(1,5):
    with open('sf_donut_shop_search_page_'+str(k)+'.htm', 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file.read(), 'html.parser')
    all = soup.find_all('div',attrs={'class':'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'})
    for i in range(2,12):
        rank = all[i].find_all('span',attrs={'class':'css-1uq0cfn'})[0]
        # ranking and store name
        name = rank.get_text()
        ranking.append(re.match('([0-9].*?)\.(.*)',name).group(1))
        store_name.append(re.match('([0-9].*?)\.(.*)',name).group(2))
        store_name = [item.replace("\xa0", "") for item in store_name]
        #url  
        url.append('https://www.yelp.com/'+rank('a')[0].get('href'))
        #star
        stars = all[i].find('span',attrs={'class':'display--inline__09f24__c6N_k border-color--default__09f24__NPAKY'}).select('div')[0].get('aria-label')
        star.append(re.match('(.*)(star rating)',stars).group(1))
        #review
        reviews = all[i].find('span',attrs={'class':'reviewCount__09f24__tnBk4 css-1e4fdj9'})
        review.append(reviews.text)
        #tag
        tags = all[i].find_all('button',attrs={'class':'css-un1vt9'})
        sub_tag = []
        for tag1 in tags:
            sub_tag.append(tag1.get_text())
        tag.append(sub_tag)
        #dollar
        if all[i].find('span',attrs={'class':'priceRange__09f24__mmOuH css-18qxe2r'}):
            d = all[i].find('span',attrs={'class':'priceRange__09f24__mmOuH css-18qxe2r'}).get_text()
            dollar.append(d)
        else:
            dollar.append('N/A')
        #deliver
        dv = all[i].find_all('div',attrs={'class':'tag__09f24__mjBHZ css-1v994a0 margin-t0-5__09f24__gboxT border-color--default__09f24__NPAKY'})
        tmp = {}
        if dv:
            for sub in dv:
                d = sub.find('path',attrs={'d':'M9.06 8l3.47-3.47a.75.75 0 00-1.06-1.06L8 6.94 4.53 3.47a.75.75 0 00-1.06 1.06L6.94 8l-3.47 3.47a.75.75 0 101.06 1.06L8 9.06l3.47 3.47a.75.75 0 001.06-1.06L9.06 8z'})
                if d:
                    tmp[sub.get_text()] = 0
                else:
                    tmp[sub.get_text()] = 1
            deliver.append(tmp)
        else:
            deliver.append('NULL')
        #order
        od = all[i].find('span',attrs={'class':'css-1enow5j'})
        if od:
            order.append('Can Deliver')
        else:
            order.append('Do not Deliver')


### Q3 Adjust your code in (2) to create a MongoDB collection called “sf_donut_shops” that stores all the extracted shop information, one document for each shop.

In [ ]:
mc = MongoClient('localhost', 27017)

In [ ]:
do3 = mc['donut']

In [ ]:
do4 = yelp_db['sf_donut_shops']

for i in range(0,40):
    inf = {
        'rank':rank[i],
        'name':name[i],
        'url':url[i],
        'rating':rating[i],
        'number of reviews':review[i],
        'tag':tag[i],
        'how many $ sign':dollar[i],
        'delivery tag etc.':deliver[i],
        'whether can be ordered on yelp':order[i]
    }
    do4.insert_one(inf)

### Q4 Using Python or Java, write a _new_ piece of code that reads all URLs stored in “sf_donut_shops” and download each shop page. Store the page to disk, “sf_donut_shop_[SR].htm” (replace [SR] with the search rank).

In [9]:
URL = "https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA&ns=1&start=0"
page1 = requests.get(URL,headers=header1)
doc1 = BeautifulSoup(page1.content, 'html.parser')

badges1 = doc1.body.findAll('div', attrs={'class': "businessName__09f24__EYSZE display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY"})


In [10]:
URL_LIST = []
for ht in badges1[2:12]:
    a = ht.find('a', attrs={'class': "css-1422juy"})
    URL_LIST.append('https://www.yelp.com/'+a['href'])

In [11]:
def main():
    try:
        for i in range(0,10):
            url = URL_LIST[i]
            r = requests.get(url,headers=header1)
            # Create a beautifulsoup object 
            soup = BeautifulSoup(r.text, 'lxml')
            with open('sf_donut_shop_%i.htm'%(i+1), 'w') as file:
                file.write(r.text)
            time.sleep(2)
    except Exception as ex:
        print("Error:" + str(ex))

if __name__ == '__main__':
    main()
                

In [128]:
#p2

In [12]:
URL = "https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA&ns=1&start=10"
page1 = requests.get(URL,headers=header1)
doc1 = BeautifulSoup(page1.content, 'html.parser')
badges1 = doc1.body.findAll('div', attrs={'class': "businessName__09f24__EYSZE display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY"})
URL_LIST = []
for ht in badges1[2:12]:
    a = ht.find('a', attrs={'class': "css-1422juy"})
    URL_LIST.append('https://www.yelp.com/'+a['href'])
 

In [13]:
def main():
    try:
        for i in range(0,10):
            url = URL_LIST[i]
            r = requests.get(url,headers=header1)
            # Create a beautifulsoup object 
            soup = BeautifulSoup(r.text, 'lxml')
            with open('sf_donut_shop_%i.htm'%(i+11), 'w') as file:
                file.write(r.text)
            time.sleep(2)
    except Exception as ex:
        print("Error:" + str(ex))

if __name__ == '__main__':
    main()

In [103]:
#3

In [14]:
URL = "https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA&ns=1&start=20"
page1 = requests.get(URL,headers=header1)
doc1 = BeautifulSoup(page1.content, 'html.parser')
badges1 = doc1.body.findAll('div', attrs={'class': "businessName__09f24__EYSZE display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY"})
URL_LIST = []
for ht in badges1[2:12]:
    a = ht.find('a', attrs={'class': "css-1422juy"})
    URL_LIST.append('https://www.yelp.com/'+a['href'])
def main():
    try:
        for i in range(0,10):
            url = URL_LIST[i]
            r = requests.get(url,headers=header1)
            # Create a beautifulsoup object 
            soup = BeautifulSoup(r.text, 'lxml')
            with open('sf_donut_shop_%i.htm'%(i+21), 'w') as file:
                file.write(r.text)
            time.sleep(2)
    except Exception as ex:
        print("Error:" + str(ex))

if __name__ == '__main__':
    main()

In [ ]:
#4

In [15]:
URL = "https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA&ns=1&start=30"
page1 = requests.get(URL,headers=header1)
doc1 = BeautifulSoup(page1.content, 'html.parser')
badges1 = doc1.body.findAll('div', attrs={'class': "businessName__09f24__EYSZE display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY"})
URL_LIST = []
for ht in badges1[2:12]:
    a = ht.find('a', attrs={'class': "css-1422juy"})
    URL_LIST.append('https://www.yelp.com/'+a['href'])
def main():
    try:
        for i in range(0,10):
            url = URL_LIST[i]
            r = requests.get(url,headers=header1)
            # Create a beautifulsoup object 
            soup = BeautifulSoup(r.text, 'lxml')
            with open('sf_donut_shop_%i.htm'%(i+31), 'w') as file:
                file.write(r.text)
            time.sleep(2)
    except Exception as ex:
        print("Error:" + str(ex))

if __name__ == '__main__':
    main()

### Q5 Using Python or Java, write new code that reads the 40 shop pages saved in (4) and parses each shop’s address, phone number, and website.

In [16]:
address = []
number = []
web = []
for i in range(1,41):
    with open("sf_donut_shop_%i.htm"%(i), encoding='UTF-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
    
    all2 = soup.find('div',attrs={'class':'css-xp8w2v padding-t2__09f24__Y6duA padding-r2__09f24__ByXi4 padding-b2__09f24__F0z5y padding-l2__09f24__kf_t_ border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-radius--regular__09f24__MLlCO background-color--white__09f24__ulvSM'})
    p = soup.find_all('p',attrs={'class':'css-na3oda'})
    if len(p) == 2:
        web.append(all2.find_all('p',attrs={'class':'css-1p9ibgf'})[0].text)
        number.append(all2.find_all('p',attrs={'class':'css-1p9ibgf'})[1].text)
    elif len(p) == 1:
        if p[0].text == 'Business website':
            web.append(all2.find('p',attrs={'class':'css-1p9ibgf'}).text)
            number.append('N/A')
        else:
            web.append('N/A')
            number.append(all2.find('p',attrs={'class':'css-1p9ibgf'}).text)
    else:
        web.append('N/A')
        number.append('N/A')
        
    address.append(soup.find('p',attrs={'class':'css-qyp8bo'}).get_text())
    

### Q6 Sign up for a free account with https://positionstack.com/ Adjust your code in (5) to query each shop address’ geolocation (long, lat). Update each shop document on the MongoDB collection “sf_donut_shops” to contain the shop’s address, phone number, website, and geolocation. Lastly, place an index on the shop’s search rank.

In [17]:
longla = []
for i in range(0,40):
    url4 = 'http://api.positionstack.com/v1/forward?access_key=88fea48dfe7281c431ba2b922f1ed775&query='+address[i]
    load = requests.get(url4, headers=header1)
    d = load.json()
    
    la = d['data'][0]['latitude']
    long = d['data'][0]['longitude']
    
    geo = (long,la)
    
    longla.append(geo)

In [28]:
data = {'ranking':ranking,'store name':store_name,'URL':url,'number of stars':star,'review number':review,'tag':tag,'number of dollar sign':dollar,'deliver option':deliver,'order online option':order,'shop’s address':address,'phone number':number,'web':web,'geolocation':longla}


In [31]:
df2 = pd.DataFrame(data)
display(df2)

,ranking,store name,URL,number of stars,review number,tag,number of dollar sign,deliver option,order online option,shop’s address,phone number,web,geolocation
0,1,Bob’s Donuts & Pastry Shop,https://www.yelp.com//biz/bobs-donuts-and-past...,4.5,3331,"[Bakeries, Donuts]",$,NULL,Do not Deliver,"1621 Polk St San Francisco, CA 94109",(415) 776-3141,http://www.bobsdonutssf.com,"(-122.421276, 37.791852)"
1,2,Uncle Benny’s Donut and Bagel,https://www.yelp.com//biz/uncle-bennys-donut-a...,4.5,389,"[Donuts, Bagels, Coffee & Tea]",$$,"{'Outdoor dining': 0, 'Delivery': 1, 'Takeout'...",Can Deliver,"2049 Irving St San Francisco, CA 94122",(628) 219-0956,N/A,"(-122.480192, 37.763299)"
2,3,Twisted Donuts and Coffee,https://www.yelp.com//biz/twisted-donuts-and-c...,4.5,508,"[Donuts, Bakeries]",$,"{'Delivery': 0, 'Takeout': 1}",Do not Deliver,"1243 Noriega St San Francisco, CA 94122",(415) 463-5437,http://twisteddonutsf.com,"(-122.477233, 37.754095)"
3,4,Bob’s Donuts,https://www.yelp.com//biz/bobs-donuts-san-fran...,4.5,118,"[Donuts, Bakeries]",$,{'Takeout': 1},Do not Deliver,"601 Baker St San Francisco, CA 94117",(415) 776-3141,https://www.bobsdonutssf.com,"(-122.441854, 37.776539)"
4,5,All Star Donuts,https://www.yelp.com//biz/all-star-donuts-san-...,4,280,"[Donuts, Sandwiches, Coffee & Tea]",$,"{'Delivery': 1, 'Takeout': 1}",Can Deliver,"901 Clement St San Francisco, CA 94118",(415) 221-9838,N/A,"(-122.468983, 37.782559)"
5,6,Mochill Mochi Donut,https://www.yelp.com//biz/mochill-mochi-donut-...,4.5,819,[Donuts],N/A,"{'Delivery': 1, 'Takeout': 1}",Can Deliver,"1737 Post St Ste 395 San Francisco, CA 94115",(650) 619-2462,https://www.mochillsf.com,"(-122.431272, 37.778008)"
6,7,Dynamo Donut & Coffee,https://www.yelp.com//biz/dynamo-donut-and-cof...,4,1945,"[Coffee & Tea, Donuts, Patisserie/Cake Shop]",$,"{'Delivery': 1, 'Takeout': 1, 'Curbside Pickup...",Do not Deliver,"2760 24th St San Francisco, CA 94110",(415) 920-1978,http://www.dynamodonut.com,"(-122.407644, 37.753022)"
7,8,Donut World,https://www.yelp.com//biz/donut-world-san-fran...,4,275,"[Coffee & Tea, Donuts]",$,{'Takeout': 1},Do not Deliver,"1399 9th Ave San Francisco, CA 94122",(415) 665-4240,N/A,"(-122.466344, 37.762306)"
8,9,Golden Donuts,https://www.yelp.com//biz/golden-donuts-san-fr...,4,125,"[Donuts, Coffee & Tea, Sandwiches]",$,{'Outdoor dining': 1},Do not Deliver,"5401 Geary Blvd San Francisco, CA 94121",(415) 422-0623,N/A,"(-122.477407, 37.780247)"
9,10,Happy Donuts,https://www.yelp.com//biz/happy-donuts-san-fra...,4,269,"[Donuts, Coffee & Tea]",$,NULL,Do not Deliver,"3801 24th St San Francisco, CA 94114",(415) 285-5890,N/A,"(-122.427598, 37.751594)"
